In [5]:
import pandas as pd
import requests
import datetime
from functions import *
import sqlite3
from dotenv import load_dotenv
import os

load_dotenv()
api_key= os.environ.get("API_KEY")

# Création d'un DataFrame avec les colonnes nécessaires
df = pd.DataFrame(columns=["ville", "temp_actuelle", "temp_ressentie", "temp_min", "temp_max", "pression_atmo", "humidité", "vitesse_vent", "dir_vent", "lever_sol", "coucher_sol"])

# création d'une liste des villes à requêter(les 20 villes)
boa = ["Gueugnon", "Paris", "Gap", "Albi", "Ajaccio", "Lyon", "Toulouse", "Poitiers", "Troyes", "Biarritz", "Beauvais", "Amiens", "Strasbourg", "Auxerre", "Marseille", "Nice", "Nantes","Dunkerque", "Dieppe","Lorient"]

url="https://api.openweathermap.org/data/2.5/weather"
#reprise de l'url de l'api openweather que l'on va requêter
for i in boa:
    params={
        "q": i,
        "appid": api_key,         #l'api_key est contenu dans le fichier .env
        "units": "metric"
    }
    
    response = requests.get(url, params=params)         #
    response_b = response.json()

    # Récupération des différents paramétres auprés de l'API, correspondant à chaque donnée 
    temp_actuelle = response_b['main']['temp']
    temp_ressentie = response_b['main']['feels_like']
    temp_min = response_b['main']['temp_min']
    temp_max = response_b['main']['temp_max']
    pression_atmo = response_b['main']['pressure']
    humidité = response_b['main']['humidity']
    vitesse_vent = response_b['wind']['speed']
    dir_vent = conversion(response_b['wind']['deg'])
    # appel à une fonction conversion (située dans functions.py)pour convertir les valeures numériques de direction du vent en "nord-est-ouest etc"
    heure_lever = datetime.datetime.fromtimestamp(response_b['sys']['sunrise'])
    heure_coucher= datetime.datetime.fromtimestamp(response_b['sys']['sunset'])
    #conversion de l'heure 
    lever_sol = heure_lever.time()
    coucher_sol = heure_coucher.time()

    # Ajout des valeurs au DataFrame créé
    df.loc[len(df)] = [i, temp_actuelle, temp_ressentie, temp_min, temp_max, pression_atmo, humidité, vitesse_vent, dir_vent, lever_sol, coucher_sol]

print(df)
# Affichage du DataFrame mis à jour
df.to_csv("meteo_villes.csv")


         ville  temp_actuelle  temp_ressentie  temp_min  temp_max  \
0     Gueugnon           7.97            5.49      7.97      8.88   
1        Paris          12.00           10.56     11.17     12.92   
2          Gap           2.04           -1.84      2.04      2.04   
3         Albi          14.14           13.36     13.28     14.14   
4      Ajaccio          15.99           14.74     15.03     15.99   
5         Lyon           8.01            7.57      7.03      8.17   
6     Toulouse          13.01           11.99     12.99     14.31   
7     Poitiers           8.23            5.32      8.23      9.58   
8       Troyes          11.03            9.39     10.43     11.03   
9     Biarritz          14.06           13.40     13.14     15.41   
10    Beauvais          11.35           10.16     11.14     12.53   
11      Amiens          11.71           10.29     11.55     12.25   
12  Strasbourg           7.90            4.18      7.14      8.49   
13     Auxerre          11.46     

In [2]:
df=pd.read_csv('tempile.csv')

conect= sqlite3.connect("bddeti.db")
#conversion du fichier csv en format sql, contenu dans la bdd "bddeti.db"

df.to_sql('villes', conect, if_exists='replace', index=False)

conect.close()
